<a href="https://colab.research.google.com/github/kartoniks/Studia/blob/master/DeepLearning/L1/assignment1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [39]:
from google.colab import files
uploaded = files.upload()

Saving train.tsv to train (1).tsv


In [55]:
import pandas as pd
import numpy as np
import scipy as sp
data = pd.read_table("train.tsv")
data.drop_duplicates("SentenceId", keep="first", inplace=True)
train_set = data[0:7000]
test_set = data[7000:]
data.head()

,PhraseId,SentenceId,Phrase,Sentiment
0,1,1,A series of escapades demonstrating the adage ...,1
63,64,2,"This quiet , introspective and entertaining in...",4
81,82,3,"Even fans of Ismail Merchant 's work , I suspe...",1
116,117,4,A positively thrilling combination of ethnogra...,3
156,157,5,Aggressive self-glorification and a manipulati...,1


In [0]:
# unique_dict = {}

# for index, row in data.iterrows():
#   if row["SentenceId"] in unique_dict.keys():
#     if unique_dict[row["SentenceId"]]["PhraseId"] > row["PhraseId"]:  
#       unique_dict[row["SentenceId"]] = row
#   else:
#     unique_dict[row["SentenceId"]] = row
# unique_data = pd.DataFrame.from_dict(unique_dict, orient='index')
# unique_data.head()

# train_set = unique_data[0:7000]
# test_set = unique_data[7000:8529]

In [41]:
import collections as co
words = co.defaultdict()

sentence_dict = co.defaultdict()
for index, row in data.iterrows():
  sentence_dict[row["Phrase"]] = row["Sentiment"]

for index, row in data.iterrows():
  for word in row["Phrase"].split(" "):
    if not word.isalpha():
      continue
    word = word.lower()
    if word not in words.keys():
      words[word] = [1, row["Sentiment"], word]     #first quantity, then sentiment
    else:
      words[word][0] += 1
      words[word][1] += row["Sentiment"]

for value in words.values():
  value[1] = value[1]/value[0]

wordList = []
for key, value in words.items():
    wordList.append(value)

wordList.sort(key = lambda x: -x[0])
wordList = wordList[0:2000]
wordList.sort(key = lambda x: -x[1])
# [9, 3.7777777777777777, 'heartwarming'] example entry

words = co.defaultdict()
id = 0
for entry in wordList:
  words[entry[2]] = entry
  words[entry[2]].append(id)
  id += 1
wordList[:10] #best words

[[10, 3.7, 'heartwarming', 0],
 [10, 3.7, 'vibrant', 1],
 [9, 3.6666666666666665, 'intoxicating', 2],
 [9, 3.6666666666666665, 'joyous', 3],
 [9, 3.6666666666666665, 'captivating', 4],
 [11, 3.6363636363636362, 'dazzling', 5],
 [9, 3.5555555555555554, 'detailed', 6],
 [14, 3.5, 'refreshing', 7],
 [10, 3.5, 'mesmerizing', 8],
 [8, 3.5, 'masterful', 9]]

In [91]:
X = [] #matrix of word counts for sentences
X_test = [] 
y = [] #vector of scores
w = len(words)
for index, row in train_set.iterrows():
  entry = np.zeros(w)
  wds = row["Phrase"].lower().split(" ")
  score = 0
  wordcount = 0
  for wd in wds:
    if wd in words:
      id = words[wd][3]
      entry[id] += 1
      score += words[wd][1] - 2
      wordcount += 1
  if wordcount != 0:
    X.append(entry)
    y.append(row["Sentiment"]) #change to score to look at average word liking

for index, row in test_set.iterrows():
  entry = np.zeros(w)
  wds = row["Phrase"].lower().split(" ")
  for wd in wds:
    if wd in words:
      id = words[wd][3]
      entry[id] += 1
  X_test.append(entry)

X = np.asarray(X)
X_test = np.asarray(X_test)
y = np.asarray(y)

print(y[0:20])

[ 0.11574559  2.400303   -0.87483316  0.66590772 -0.5490727   0.9882401
 -2.06800104  0.10919108  0.23101599 -1.32002173 -2.56082155  0.26254311
  1.73922044  4.44224143 -0.49461068  2.53813514  1.92419261  2.27162579
 -0.7222886  -0.41119261]


In [0]:
################################################### Negative encoding #######################################################
X = [] #matrix of word counts for sentences
X_test = [] 
y = [] #vector of scores

w = len(words)
for index, row in train_set.iterrows():
  entry = np.zeros(w)
  wds = row["Phrase"].split(" ")
  score = 0
  wordcount = 0
  for i in range(len(wds)):
    wd = wds[i]
    if wd in words:
      if wd in ["not", "n't"] and i < len(wds)-1:
        next_wd = wds[i+1]
        if next_wd in words:
          id = words[next_wd][3]
          entry[id] -= 2  #because next iteration will increase
          score -= 2*words[next_wd][1]
        else:
          continue
      else:
        id = words[wd][3]
        entry[id] += 1
      score += words[wd][1]
      wordcount += 1
  if wordcount != 0:
    X.append(entry)
    y.append(row["Sentiment"])

for index, row in test_set.iterrows():
  entry = np.zeros(w)
  wds = row["Phrase"].split(" ")
  wordcount = 0
  for i in range(len(wds)):
    wd = wds[i]
    if wd in words:
      if wd in ["not", "n't"] and i < len(wds)-1:
        next_wd = wds[i+1]
        if next_wd in words:
          id = words[next_wd][3]
          entry[id] -= 2  #because next iteration will increase
        else:
          continue
      else:
        id = words[wd][3]
        entry[id] += 1
      wordcount += 1
  X_test.append(entry)

X = np.asarray(X)
X_test = np.asarray(X_test)
y = np.asarray(y)

In [0]:
################################################### Negative and emphasis encoding #######################################################
X = [] #matrix of word counts for sentences
X_test = [] 
y = [] #vector of scores
neg = -2
emph= 2

w = len(words)
for index, row in train_set.iterrows():
  entry = np.zeros(w)
  wds = row["Phrase"].split(" ")
  score = 0
  wordcount = 0
  for i in range(len(wds)):
    wd = wds[i]
    if wd in words:
      if wd in ["not", "n't"] and i < len(wds)-1:
        next_wd = wds[i+1]
        if next_wd in ["very", "really"]:
          next_wd = wds[i+2] # jump over very
          if next_wd not in words:
            continue
          id = words[next_wd][3]
          entry[id] += neg*emph
          score -= 4*words[next_wd][1]
        elif next_wd in words:
          id = words[next_wd][3]
          entry[id] += neg  #because next iteration will increase
          score -= 2*words[next_wd][1]
      elif wd in ["very", "really"]:
        next_wd = wds[i+1]
        if next_wd in words:
          id = words[next_wd][3]
          entry[id] += emph  #because next iteration will increase
          score += 4*words[next_wd][1]
      else:
        id = words[wd][3]
        entry[id] += 1
      score += words[wd][1]
      wordcount += 1
  if wordcount != 0:
    X.append(entry)
    y.append(row["Sentiment"])

for index, row in test_set.iterrows():
  entry = np.zeros(w)
  wds = row["Phrase"].split(" ")
  wordcount = 0
  for i in range(len(wds)):
    wd = wds[i]
    if wd in words:
      if wd in ["not", "n't"] and i < len(wds)-1:
        next_wd = wds[i+1]
        if next_wd in ["very", "really"]:
          next_wd = wds[i+2] # jump over very
          if next_wd not in words:
            continue
          id = words[next_wd][3]
          entry[id] += neg*emph
        elif next_wd in words:
          id = words[next_wd][3]
          entry[id] += neg
      elif wd in ["very", "really"]:
        next_wd = wds[i+1]
        if next_wd in words:
          id = words[next_wd][3]
          entry[id] += emph 
      else:
        id = words[wd][3]
        entry[id] += 1
      wordcount += 1
  # if wordcount != 0:
  X_test.append(entry)

X = np.asarray(X)
X_test = np.asarray(X_test)
y = np.asarray(y)

In [92]:
from sklearn.linear_model import LogisticRegression

y_int = np.zeros(len(y))
for i in range(len(y)):
  y_int[i] = round(y[i])
logisticRegr = LogisticRegression(random_state=0, max_iter=50)
logisticRegr.fit(X, y_int)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=50,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=0, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [93]:
prediction = logisticRegr.predict(X_test)
displayData = []

ind = 0
for index, row in test_set.iterrows():
  entry = [prediction[ind], row["Phrase"]]
  displayData.append(entry)
  ind += 1
displayData.sort(key = lambda x: -x[0])

def threshold(data): #maps data to correct scores
  limit = 4
  maxData = 0
  for d in data:
    maxData = max(maxData, d[0])
  for d in data:
    d[0] = int(round(d[0]*limit/maxData))

def rescale(data):
  data.sort(key = lambda x: x[0])
  print(data[:10])
  chances = [0,0,0,0,0]
  entry_count = len(test_set)
  for index, row in test_set.iterrows():
    chances[row["Sentiment"]] += 1
  print(chances)
  for i in range(len(chances)):
    if i != 0:
      chances[i] += chances[i-1]
  print(chances)
  sent = 0    #increases when reached a certain percentage of data, same as in train set
  for i in range(len(data)):
    d = data[i]
    d[0] = sent
    if i > chances[sent]:
      sent += 1


rescale(displayData)
# threshold(displayData)
displayData[0:10]


[[-5.0, "One thing 's for sure -- if George Romero had directed this movie , it would n't have taken the protagonists a full hour to determine that in order to kill a zombie you must shoot it in the head ."], [-5.0, 'Girlfriends are bad , wives are worse and babies are the kiss of death in this bitter Italian comedy .'], [-4.0, 'too many scenarios in which the hero might have an opportunity to triumphantly sermonize , and too few that allow us to wonder for ourselves if things will turn out okay .'], [-4.0, 'The makers of Mothman Prophecies succeed in producing that most frightening of all movies -- a mediocre horror film too bad to be good and too good to be bad .'], [-3.0, 'The gags that fly at such a furiously funny pace that the only rip off that we were aware of was the one we felt when the movie ended so damned soon .'], [-3.0, 'MIB II is a movie that makes it possible for the viewer to doze off for a few minutes or make several runs to the concession stand and\\/or restroom and 

[[0,
  "One thing 's for sure -- if George Romero had directed this movie , it would n't have taken the protagonists a full hour to determine that in order to kill a zombie you must shoot it in the head ."],
 [0,
  'Girlfriends are bad , wives are worse and babies are the kiss of death in this bitter Italian comedy .'],
 [0,
  'too many scenarios in which the hero might have an opportunity to triumphantly sermonize , and too few that allow us to wonder for ourselves if things will turn out okay .'],
 [0,
  'The makers of Mothman Prophecies succeed in producing that most frightening of all movies -- a mediocre horror film too bad to be good and too good to be bad .'],
 [0,
  'The gags that fly at such a furiously funny pace that the only rip off that we were aware of was the one we felt when the movie ended so damned soon .'],
 [0,
  'MIB II is a movie that makes it possible for the viewer to doze off for a few minutes or make several runs to the concession stand and\\/or restroom and n

In [94]:
def check(predictions): #[2, 'At 78 minutes it just zings along with vibrance and warmth .']]
  good, bad = 0, 0
  for p in predictions:
    if p[0] != int(sentence_dict[p[1]]):
      bad += 1
    else:
      good += 1
  return (good/(bad+good))
  
check(displayData) #accuracy of prediction


0.31131458469587964